In [1]:
import requests
import time

headers = {"user-agent" : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36"}

# request package를 사용하여 error handling
def download(method, url, param=None, data=None, timeout=1, maxretries=3):
    try:
        resp = requests.request(method, url, params=param, data=data, headers=headers)
        # 한글처럼 아스키가 아닌 문자들을 변환시킬 때 서비스키도 같이 바꿔버림
        resp.raise_for_status()
        #if(resp.headers['Content-Type'].split("/")[0] == "image"):
            #resp.headers['Content-Type'].split("/")[1] #확장자
    except requests.exceptions.HTTPError as e:
        if 500 <= e.response.status_code < 600 and maxretries > 0:
            print(e.response.status_code)
            print(e.response.reason)
            time.sleep(timeout) # 1초 정도 기다림
            print(maxretries)
            print("재시도") #maxretries
            download(method, url, param, data, timeout, maxretries-1)
        else:
            print(e.response.status_code)
            print(e.response.reason)
    return resp

In [2]:
url = "http://example.webscraping.com/places/default/index"
html = download("get", url)

In [3]:
from bs4 import BeautifulSoup
dom = BeautifulSoup(html.text, "lxml")
[_["href"] for _ in dom.find_all("a")]

['#',
 '/places/default/user/register?_next=/places/default/index',
 '/places/default/user/login?_next=/places/default/index',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Afghanistan-1',
 '/places/default/view/Aland-Islands-2',
 '/places/default/view/Albania-3',
 '/places/default/view/Algeria-4',
 '/places/default/view/American-Samoa-5',
 '/places/default/view/Andorra-6',
 '/places/default/view/Angola-7',
 '/places/default/view/Anguilla-8',
 '/places/default/view/Antarctica-9',
 '/places/default/view/Antigua-and-Barbuda-10',
 '/places/default/index/1']

다시 http가 붙은 full url 형태로 바꿔줘야 함 <- urljoin  
\# : 실제 내용을 담고 있는 애가 아님! 이런 애들 만나면 피해야 함

In [4]:
[_["href"] for _ in dom.find_all("a") if len(_["href"])>3] # #을 없앰

['/places/default/user/register?_next=/places/default/index',
 '/places/default/user/login?_next=/places/default/index',
 '/places/default/index',
 '/places/default/search',
 '/places/default/view/Afghanistan-1',
 '/places/default/view/Aland-Islands-2',
 '/places/default/view/Albania-3',
 '/places/default/view/Algeria-4',
 '/places/default/view/American-Samoa-5',
 '/places/default/view/Andorra-6',
 '/places/default/view/Angola-7',
 '/places/default/view/Anguilla-8',
 '/places/default/view/Antarctica-9',
 '/places/default/view/Antigua-and-Barbuda-10',
 '/places/default/index/1']

In [5]:
[requests.compat.urljoin(url,_["href"]) for _ in dom.find_all("a") if len(_["href"])>3]

['http://example.webscraping.com/places/default/user/register?_next=/places/default/index',
 'http://example.webscraping.com/places/default/user/login?_next=/places/default/index',
 'http://example.webscraping.com/places/default/index',
 'http://example.webscraping.com/places/default/search',
 'http://example.webscraping.com/places/default/view/Afghanistan-1',
 'http://example.webscraping.com/places/default/view/Aland-Islands-2',
 'http://example.webscraping.com/places/default/view/Albania-3',
 'http://example.webscraping.com/places/default/view/Algeria-4',
 'http://example.webscraping.com/places/default/view/American-Samoa-5',
 'http://example.webscraping.com/places/default/view/Andorra-6',
 'http://example.webscraping.com/places/default/view/Angola-7',
 'http://example.webscraping.com/places/default/view/Anguilla-8',
 'http://example.webscraping.com/places/default/view/Antarctica-9',
 'http://example.webscraping.com/places/default/view/Antigua-and-Barbuda-10',
 'http://example.webscr

전체주소 생성

In [13]:
def parseURL(seed):
    urlList = list()
    
    # 실 데이터를 다루면 태그가 a로 시작하더라도 href가 없을 수도 있음
    
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url,_["href"]) 
     for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"])>3]

In [10]:
queue = list()
queue.append("http://example.webscraping.com/places/default/index")
seen = list() #기존에 방문한 횟수

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))
#    queue.extend(linkList)
    # 이렇게 돌리면 안됨
    # 첫 페이지에 들어갔을 때 메뉴 혹은 로그인 등은 다른 페이지에 가도 동일하게 있음
    # 즉, linkList에 계속해서 중복되는 url이 생길 것!

429
TOO MANY REQUESTS
Queue: 0, Seen: 1


In [15]:
queue = list()
queue.append("https://www.google.com")
seen = list() #기존에 방문한 횟수

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 38, Seen: 1
Queue: 41, Seen: 2
Queue: 65, Seen: 3


KeyboardInterrupt: 

In [88]:
q = '박보영'
param = {
    'source': 'hp',
    'ei': '1jUoXe7RGoqV8wW3gJSwCg',
    'q': q,
    'gs_l': 'psy-ab.3..35i39j0l9.6818.8611..11888...4.0..1.114.1031.4j6......0....1..gws-wiz.....10..0i131.zx4IVFyfgqM'
        }
resp = download("get", "https://www.google.com/search", param)
dom = BeautifulSoup(resp.text, 'lxml')
googlesearchurl = [_["href"]  for _ in dom.select(".rc> .r a[href^=http]")]
  

In [89]:
googlesearchurl

['https://namu.wiki/w/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81',
 'https://webcache.googleusercontent.com/search?q=cache:gv9t4wBBoSMJ:https://ko.wikipedia.org/wiki/%25EB%25B0%2595%25EB%25B3%25B4%25EC%2598%2581+&cd=12&hl=ko&ct=clnk&gl=kr',
 'https://ko.wikipedia.org/wiki/%EB%B0%95%EB%B3%B4%EC%98%81%EC%9D%98_%EC%9E%91%ED%92%88_%EB%AA%A9%EB%A1%9D',
 'https://webcache.googleusercontent.com/search?q=cache:VhhxhBJ2kukJ:https://ko.wikipedia.org/wiki/%25EB%25B0%2595%25EB%25B3%25B4%25EC%2598%2581%25EC%259D%2598_%25EC%259E%2591%25ED%2592%2588_%25EB%25AA%25A9%25EB%25A1%259D+&cd=13&hl=ko&ct=clnk&gl=kr',
 'https://news.joins.com/article/22895953',
 'https://webcache.googleusercontent.com/search?q=cache:C8ZtkDOVE_cJ:https://news.joins.com/article/22895953+&cd=14&hl=ko&ct=clnk&gl=kr',
 'https://movie.daum.net/person/main?personId=105244',
 'https://webcache.googleusercontent.com/search?q=cache:ahd-t4CdgoAJ:https://movie.daum.net/person/main%3FpersonId%

In [18]:
queue = list()
for link in googlesearchurl:
    queue.append(link)
seen = list() #기존에 방문한 횟수

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 871, Seen: 1
Queue: 873, Seen: 2
Queue: 1132, Seen: 3


KeyboardInterrupt: 

In [27]:
q = '박보영'
param = {
    'sm' : 'tab_hty',
    'fbm': 1,
    'ie' : 'utf8',
    'query' : q
        }
resp = download("get", "https://search.naver.com/search.naver", param)

dom = BeautifulSoup(resp.text, 'lxml')

<!DOCTYPE html>
<html lang="ko"> <head> <meta charset="utf-8"/> <meta content="always" name="referrer"/> <meta content="telephone=no,address=no,email=no" name="format-detection"/> <meta content="width=device-width,initial-scale=1.0,maximum-scale=2.0" name="viewport"/> <meta content="박보영 : 네이버 통합검색" property="og:title"/> <meta content="https://ssl.pstatic.net/sstatic/search/common/og_v3.png" property="og:image"/> <meta content="'박보영'의 네이버 통합검색 결과입니다." property="og:description"/> <meta content="'박보영'의 네이버 통합검색 결과입니다." lang="ko" name="description"/> <title>박보영 : 네이버 통합검색</title> <link href="https://ssl.pstatic.net/sstatic/search/favicon/favicon_140327.ico" rel="shortcut icon"/> <link href="https://ssl.pstatic.net/sstatic/search/opensearch-description.https.xml" rel="search" title="Naver" type="application/opensearchdescription+xml"/><link href="https://ssl.pstatic.net/sstatic/search/pc/css/search1_190711.css" rel="stylesheet" type="text/css"/> <link href="https://ssl.pstatic.net/sstatic/s

In [35]:
naverlist = [_["href"] for _ in dom.select("a.sh_blog_title._sp_each_url._sp_each_title")]

In [30]:
requests.compat.urlparse("https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950")

ParseResult(scheme='https', netloc='blog.naver.com', path='/imagetech1', params='', query='Redirect=Log&logNo=221530358950', fragment='')

In [32]:
# focused crawling
def checkBlog(url):
    return requests.compat.urlparse(url)[1] == "blog.naver.com"

In [33]:
checkBlog("https://blog.naver.com/imagetech1?Redirect=Log&logNo=221530358950")

True

In [48]:
def parseURL(seed):
    # 실 데이터를 다루면 태그가 a로 시작하더라도 href가 없을 수도 있음
    
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    if len(dom.select("#mainFrame")) < 1:
        return []
    
    seed = requests.compat.urljoin(seed, dom.select("#mainFrame")[0]["src"])
    
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(url,_["href"]) 
     for _ in dom.find_all("a") 
            if _.has_attr("href") and len(_["href"])>3
           and checkBlog(requests.compat.urljoin(seed, _["href"]))]

In [50]:
queue = list()
for link in naverlist:
    queue.append(link)

seen = list() #기존에 방문한 횟수

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 19, Seen: 1
Queue: 18, Seen: 2
Queue: 17, Seen: 3
Queue: 29, Seen: 4
Queue: 42, Seen: 5
Queue: 41, Seen: 6
Queue: 40, Seen: 7
Queue: 39, Seen: 8
Queue: 38, Seen: 9
Queue: 37, Seen: 10
Queue: 36, Seen: 11
Queue: 35, Seen: 12
Queue: 34, Seen: 13
Queue: 33, Seen: 14


KeyboardInterrupt: 

In [67]:
q = '박보영'
param = {
    'w':'tot',
    'DA':'YZR',
    't__nil_searchbox':'btn',
    'q' : q
        }
resp = download("get", "https://search.daum.net/search", param)
dom = BeautifulSoup(resp.text, 'lxml')
dom

<!DOCTYPE html>
<html class="mac chrome" lang="ko" xmlns="http://www.w3.org/1999/xhtml">
<head profile="http://a9.com/-/spec/opensearch/1.1/">
<meta content="text/html;charset=utf-8" http-equiv="content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="off" name="autocomplete"/>
<meta content="always" name="referrer"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="박보영 – Daum 검색" property="og:title"/>
<meta content="https://search.daum.net/search?w=tot&amp;q=%EB%B0%95%EB%B3%B4%EC%98%81" property="og:url"/>
<meta content="Daum 검색에서 박보영에 대한 최신정보를 찾아보세요." property="og:description"/>
<meta content="https://search1.daumcdn.net/search/statics/common/img/og_search.png" property="og:image"/>
<meta content="다음검색" property="og:site_name"/>
<title>박보영 – Daum 검색</title>
<link href="//search.daum.net/OpenSearch.xml" rel="search" title="Daum" type="application/opensearchdescription+xml"/>
<link charset="utf-8" href="//search1.daumcdn.net/search/stat

In [71]:
import re
def checktistory(url):
    return requests.compat.urlparse(url)[1].endswith("tistory.com")

In [70]:
[(_["href"]) for _ in dom.select("#blogColl a.f_link_b") 
            if checktistory(_["href"])]

http://adam24eve.tistory.com/858
http://cscs0717.tistory.com/27
http://listup.tistory.com/248
http://34344541.tistory.com/43


['http://adam24eve.tistory.com/858',
 'http://cscs0717.tistory.com/27',
 'http://listup.tistory.com/248',
 'http://34344541.tistory.com/43']

In [77]:
def parseURL(seed):
    # 실 데이터를 다루면 태그가 a로 시작하더라도 href가 없을 수도 있음
    
    html = download("get", seed[0])
    dom = BeautifulSoup(html.text, "lxml")
    
    return [(requests.compat.urljoin(seed[0],_["href"]), seed[1]+1) 
     for _ in dom.find_all("a") 
            if seed[1] < 2 and _.has_attr("href") and len(_["href"])>3
           and checktistory(requests.compat.urljoin(seed[0], _["href"]))]

In [78]:
queue = list()
queue.extend([(_["href"],1)
             for _ in dom.select("#blogColl a.f_link_b")
             if checktistory(_["href"])])

seen = list() #기존에 방문한 횟수

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))

Queue: 45, Seen: 1
Queue: 61, Seen: 2
Queue: 85, Seen: 3
Queue: 92, Seen: 4
Queue: 91, Seen: 5
Queue: 90, Seen: 6
Queue: 89, Seen: 7
Queue: 88, Seen: 8
Queue: 87, Seen: 9
Queue: 86, Seen: 10
Queue: 85, Seen: 11
Queue: 84, Seen: 12
Queue: 83, Seen: 13
Queue: 82, Seen: 14
Queue: 81, Seen: 15
Queue: 80, Seen: 16
Queue: 79, Seen: 17
Queue: 78, Seen: 18
Queue: 77, Seen: 19
Queue: 76, Seen: 20
Queue: 75, Seen: 21
Queue: 74, Seen: 22
Queue: 73, Seen: 23
Queue: 72, Seen: 24
Queue: 71, Seen: 25
Queue: 70, Seen: 26
Queue: 69, Seen: 27
Queue: 68, Seen: 28
Queue: 67, Seen: 29
Queue: 66, Seen: 30
Queue: 65, Seen: 31
Queue: 64, Seen: 32
Queue: 63, Seen: 33
Queue: 62, Seen: 34
Queue: 61, Seen: 35
Queue: 60, Seen: 36
Queue: 59, Seen: 37
Queue: 58, Seen: 38
Queue: 57, Seen: 39
Queue: 56, Seen: 40
Queue: 55, Seen: 41
Queue: 54, Seen: 42
Queue: 53, Seen: 43
Queue: 52, Seen: 44
Queue: 51, Seen: 45
Queue: 50, Seen: 46
Queue: 49, Seen: 47
Queue: 48, Seen: 48
Queue: 47, Seen: 49
Queue: 46, Seen: 50
Queue: 45

In [79]:
re.search(".+?\.tistory.com", url)

[('http://adam24eve.tistory.com/858', 1),
 ('http://cscs0717.tistory.com/27', 1),
 ('http://listup.tistory.com/248', 1),
 ('http://34344541.tistory.com/43', 1),
 ('http://adam24eve.tistory.com/category', 2),
 ('http://adam24eve.tistory.com/category/Cosmos', 2),
 ('http://adam24eve.tistory.com/category/Mystery%20world', 2),
 ('http://adam24eve.tistory.com/category/Star', 2),
 ('http://adam24eve.tistory.com/category/Political%20', 2),
 ('http://adam24eve.tistory.com/category/It-game', 2),
 ('http://adam24eve.tistory.com/guestbook', 2),
 ('https://adam24eve.tistory.com/rss', 2),
 ('http://adam24eve.tistory.com/858', 2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91', 2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%EB%82%98%EC%9D%B4',
  2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%EB%B0%95%EB%B3%B4%EC%98%81%20%EC%97%B4%EC%95%A0',
  2),
 ('http://adam24eve.tistory.com/tag/%EA%B9%80%EC%98%81%EA%B4%91%20%ED%82%A4',
  2),
 ('ht

## DB 붙이기  
queue와 반대되는 개념 : stack?  
URL -seen인지 check하는 애가 있어야 함  
최소 네 개의 column으로 구성된 스키마를 하나 생성해야 함  
=> id, url, seen, date  
TABLE 하나를 가지고 크롤링을 하겠다!  
만약 focused 크롤링을 하고 싶다면 url column을 건드려야 함  
table1 = id(pk), path, param, seen, date  
table2 = id(pk), netloc, date <-focused crawling을 하고 싶다면 이 테이블만 관리하면 됨 netloc에 특정 도메인만 써 주면 됨. 제대로 관리를 하려면 여기에도 date가 있어야 함.  
url = netloc + path + param

In [81]:
import sqlite3

In [139]:
con = sqlite3.connect("bot.db")
cur = con.cursor()

In [140]:
cur.executescript("""
    DROP TABLE IF EXISTS table1;
    CREATE TABLE table1(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        table2_id INTEGER NOT NULL,
        path TEXT NOT NULL,
        param TEXT,
        depth INTEGER NOT NULL,
        inbound INTEGER NOT NULL,
        seen BOOLEAN DEFAULT FALSE NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
    
    DROP TABLE IF EXISTS table2;
    CREATE TABLE table2(
        id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
        netloc TEXT NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL
    );
""")
con.commit()
#inbound를 만드는 이유 : 어떤 페이지가 중요한지 가중치를 계산할 수 있음 (pageRank계산)

In [127]:
def parseURL(seed):
    html = download("get", seed)
    dom = BeautifulSoup(html.text, "lxml")
    
    return [requests.compat.urljoin(seed, _["href"]) 
     for _ in dom.find_all("a") if _.has_attr("href") and len(_["href"])>3]

In [141]:
# Seed URLs -> DB Insert
url = "https://www.google.com/search"
resp = download("get", url, {"q":"박보영"})
dom = BeautifulSoup(resp.text, 'lxml')

for href in [_.find_parent()["href"]
            for _ in dom.select(".LC20lb")]:
    _urlparse = requests.compat.urlparse(href)
    netloc = "://".join(_urlparse[:2])
    cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
    netlocID = cur.fetchone()
    if not netlocID:
        cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
        con.commit()
        # LIMIT 0,1 : select문으로 최대 하나만 받아옴
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
    cur.execute("""
        INSERT INTO table1(table2_id, path, param, depth, inbound) 
        VALUES(?, ?, ?, ?, ?)
        """, [netlocID[0], _urlparse[2], _urlparse[4], 1, 0])
    con.commit()
    print(cur.lastrowid, netlocID)
# DB Select Limit 0,1

1 (1,)
2 (2,)
3 (2,)
4 (3,)
5 (4,)
6 (5,)
7 (6,)
8 (7,)
9 (8,)


In [142]:
while True:
    cur.execute("""
        SELECT table1.id, table2.netloc, 
                table1.path, table1.param, 
                table1.depth, table2.id
        FROM table1 
        JOIN table2 
            ON table1.table2_id=table2.id 
        WHERE table1.seen=FALSE and table1.depth < 2
        ORDER BY table1.date ASC 
        LIMIT 0, 1;
    """)
    seed = cur.fetchone()
    if not seed:
        break
    cur.execute("""
        UPDATE table1
        SET seen=TRUE
        WHERE id=?
    """, [seed[0]])
    con.commit()
    
    baseURL = "{0}{1}?{2}".format(seed[1], seed[2], seed[3])
    # 여기까지 하면 실제 queue에서 baseURL 구문을 하나 가져온 것과 같음
    for href in parseURL(baseURL):
        _urlparse = requests.compat.urlparse(href)
        netloc = "://".join(_urlparse[:2])
        cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
        netlocID = cur.fetchone()
        if not netlocID:
            cur.execute("INSERT INTO table2(netloc) VALUES(?)", [netloc])
            con.commit()
            cur.execute("SELECT id FROM table2 WHERE netloc=? LIMIT 0,1", [netloc])
            netlocID = cur.fetchone()
        cur.execute("""
        INSERT INTO 
        table1(table2_id, path, param, depth, inbound) 
        VALUES(?, ?, ?, ?, ?)""", 
                    [netlocID[0], _urlparse[2], _urlparse[4], seed[4]+1, seed[5]])
        con.commit()
#         i += 1
#         if i > 1000:
#             break
#         print(cur.lastrowid, netlocID)
        
#     print(seed)
#     print("{0}{1}?{2}".format(seed[1], seed[2], seed[3]))
#     break

In [159]:
cur.execute("""
    select inbound, count(inbound)
    from table1
    where table2_id=27 group by inbound;
""")
cur.fetchall()

[(1, 47), (2, 1)]

In [161]:
cur.execute("""
    select count(inbound)
    from table1
    where inbound=1
""")
cur.fetchall()

[(1322,)]

1번 사이트의 27번 사이트에 대한 영향력은 (47/1322)*weight  
1322는 1번 사이트에 inbound되어 있는 링크 개수,  
47은 1번 사이트로부터 27번 사이트가 inbound되어 있는 개수  

In [138]:
cur.close()
con.close()

In [104]:
requests.compat.urlparse(html.request.url)

ParseResult(scheme='http', netloc='example.webscraping.com', path='/places/default/index', params='', query='', fragment='')

params는 post로 넘길 때

In [ ]:
queue = list()
queue.extend([(_.find_parent()["href"], 1)
             for _ in dom.select(".LC201b")])
seen = list() #기존에 방문한 횟수

while queue:
    baseURL = queue.pop(0)
    seen.append(baseURL)
    time.sleep(1)
    linkList = parseURL(baseURL)
    for link in linkList:
        if link not in queue and link not in seen:
            queue.append(link)
    print("Queue: {0}, Seen: {1}".format(len(queue), len(seen)))